In [2]:
import os
import json
from dotenv import load_dotenv
import torch
import time
import math, random
from pymongo import MongoClient
from bson.objectid import ObjectId
from pymongo.server_api import ServerApi
from pinecone import Pinecone

## Load Config
with open('config/videos.json') as config_file:
    videos = json.load(config_file)
with open('config/name_to_url.json') as config_file:
    name_to_url = json.load(config_file)
with open('config/alltopics.json') as config_file:
    alltopics = json.load(config_file)

load_dotenv(dotenv_path=".env")

CUR_DIR = os.getcwd()
pc = Pinecone(api_key=os.environ["PINECONE_KEY"])
index = pc.Index("pretechnigala")
DB_NAME = "preTechnigalaClean_db"
COLLECTION_NAME = "video_metadata"
MONGO_DB_CLIENT = MongoClient(os.getenv("MONGODB_URI"), server_api=ServerApi('1'))
OUTPUT_DIR = f"{CUR_DIR}/data/topics"
QUERY_MODE = "avg"  
VECTOR_MODE = "avg"

with open('config/videos.json') as config_file:
    VIDEOS = json.load(config_file)
with open('config/name_to_url.json') as config_file:
    NAME_TO_URL = json.load(config_file)
with open('config/alltopics.json') as config_file:
    ALLTOPICS = json.load(config_file)

/Users/klinalb/anaconda3/envs/CS98/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
from pinecone import Pinecone

CUR_DIR = os.getcwd()
pc = Pinecone(api_key=os.environ["PINECONE_KEY"])
index = pc.Index("pretechnigala")
DB_NAME = "preTechnigalaClean_db"
COLLECTION_NAME = "video_metadata"
MONGO_DB_CLIENT = MongoClient(os.getenv("MONGODB_URI"), server_api=ServerApi('1'))
OUTPUT_DIR = f"{CUR_DIR}/data/topics"
QUERY_MODE = "avg"  
VECTOR_MODE = "avg"


In [4]:
# fetch all vectors with matching metadata
index_to_ids = {}
for i, name in ALLTOPICS.items():
    ## find index in field "topicIds", which is an array
    print(i)
    docs = MONGO_DB_CLIENT[DB_NAME][COLLECTION_NAME].find({"topicId": {"$in": [int(i)]}})
    for doc in docs:
        if i not in index_to_ids:
            index_to_ids[i] = []
        index_to_ids[i].append(str(doc["_id"]))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72


In [5]:
## Compute average tensor

for i, ids in index_to_ids.items():
    avg_ids = [f'{id}_avg' for id in ids]
    avg = index.fetch(ids=avg_ids)
    vectors = []
    for id, obj in avg['vectors'].items():
        vectors.append(obj['values'])
    
    print(len(vectors))
    avg_tensor = torch.tensor(vectors).mean(dim=0)
    max_tensor = torch.tensor(vectors).max(dim=0)
    print(avg_tensor)
    torch.save(avg_tensor, f"{OUTPUT_DIR}/{i}.pt")


42
tensor([ 0.1743,  3.6719,  0.9169,  ..., -2.2407,  0.1335, -0.2054])
2
tensor([ 0.2462,  3.4150,  0.8492,  ..., -1.8569, -0.5530, -0.5791])
5
tensor([ 0.0632,  3.0648,  0.6177,  ..., -2.4754,  0.9071,  0.4283])
4
tensor([ 0.2973,  4.0522,  0.8009,  ..., -1.8320,  0.1287,  0.0906])
6
tensor([-0.0167,  3.5537,  0.9373,  ..., -2.4419, -0.0089, -0.7463])
5
tensor([ 0.5119,  4.0145,  1.2452,  ..., -2.3480,  0.2022, -0.4651])
65
tensor([-0.1117,  2.8815,  0.6607,  ..., -1.9972,  0.2921, -0.4062])
8
tensor([-0.2706,  3.0547,  0.5089,  ..., -1.9198,  0.6205, -0.4594])
5
tensor([ 0.0799,  2.8836,  0.4850,  ..., -2.5594, -0.1081, -0.8728])
7
tensor([ 0.1213,  2.9275,  0.7177,  ..., -0.9195,  0.6304, -0.1862])
6
tensor([ 0.1260,  3.2021,  0.5320,  ..., -2.1325,  0.1852, -0.3781])
4
tensor([-0.3290,  2.9712,  0.6461,  ..., -2.0898,  0.6927, -0.2776])
49
tensor([-0.0476,  3.7432,  0.5252,  ..., -2.8429, -0.1795, -0.5300])
6
tensor([-0.0860,  3.9089,  0.9856,  ..., -3.2998, -0.7077, -0.5741])
5
t

In [8]:
index = pc.Index("pretechnigala-topics")

vectors = []
for embedding_file in os.listdir(OUTPUT_DIR):
    print(torch.load(f"{OUTPUT_DIR}/{embedding_file}"))
    tensor = torch.load(f"{OUTPUT_DIR}/{embedding_file}").to('cpu').numpy().tolist()
    id = embedding_file.split(".")[0]
    vectors.append({"values": tensor, "id": f'{id}_{VECTOR_MODE}'})

index.upsert(vectors)

tensor([-0.2831,  3.2785,  0.7567,  ..., -1.9876, -0.1620, -0.5045])
tensor([ 0.1760,  3.3135,  1.3129,  ..., -2.5718, -0.2189,  0.3574])
tensor([ 0.0991,  3.3292,  0.6228,  ..., -2.5833,  0.3006, -0.4889])
tensor([-0.1784,  4.4459,  0.3328,  ..., -2.3697, -0.2008,  0.3597])
tensor([ 0.4392,  2.8523,  0.8381,  ..., -2.2512, -0.4941, -0.8894])
tensor([ 0.1987,  2.5028,  1.2663,  ..., -2.5322,  0.0290, -0.0066])
tensor([ 0.2973,  4.0522,  0.8009,  ..., -1.8320,  0.1287,  0.0906])
tensor([ 0.1241,  3.3503,  0.9824,  ..., -2.7852,  0.2414, -0.3670])
tensor([-0.2182,  2.6770,  0.4567,  ..., -3.0379, -0.4566, -0.7693])
tensor([ 0.1780,  2.9747,  1.1287,  ..., -2.2983,  0.2917, -0.2317])
tensor([-0.0167,  3.5537,  0.9373,  ..., -2.4419, -0.0089, -0.7463])
tensor([ 0.1873,  4.4014,  0.8160,  ..., -2.7319,  0.4154, -0.3413])
tensor([ 0.0635,  3.3431,  1.1436,  ..., -2.2464, -0.0525, -0.4865])
tensor([-0.6772,  3.0347,  0.3349,  ..., -1.9607, -0.1371, -0.0677])
tensor([ 0.1743,  3.6719,  0.9169,

{'upserted_count': 72}